**Imports**

In [1]:
import torch

In [2]:
import torch
import torch.nn as nn
from torch_geometric.datasets import Planetoid

**Dataset**

In [3]:

dataset = Planetoid(root="data", name="Cora")
data = dataset[0]


In [4]:
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [7]:
len(data.train_mask[data.train_mask == True])

140

In [12]:
len(data.train_mask[data.val_mask == True])

500

In [10]:
data.y.unique()

tensor([0, 1, 2, 3, 4, 5, 6])

**Building Adjacency Matrix**

In [11]:
N = data.num_nodes 
edge_index =  data.edge_index

A = torch.zeros((N,N))
A[edge_index[0], edge_index[1]] = 1
A = A + A.T
A[A > 1] = 1
A[A == 1].shape

torch.Size([10556])

In [12]:
I = torch.eye(N)
A_hat = A + I # adding self loops

D_hat = torch.diag(A_hat.sum(dim = 1))
D_hat_inv_sqrt = torch.linalg.inv(torch.sqrt(D_hat))

A_norm = D_hat_inv_sqrt @ A_hat @ D_hat_inv_sqrt

**GCN Model**

In [20]:
class GCN(nn.Module):
  def __init__(self, in_dim, hidden_dim, out_dim):
    super().__init__()
    self.W1 = torch.nn.Parameter(torch.randn(in_dim, hidden_dim))
    self.W2 = torch.nn.Parameter(torch.randn(hidden_dim, hidden_dim))
    self.W3 = torch.nn.Parameter(torch.randn(hidden_dim, out_dim))

  def forward(self, A_norm , X):
    H = torch.relu(A_norm @ X @ self.W1) # 1st Aggreation and projection
    H = A_norm @ H @ self.W2 # 2nd Aggregation & projection
    H = A_norm @ H @ self.W3
    return H
  


In [16]:
A_norm.shape

torch.Size([2708, 2708])

**Train Loop**

In [33]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

model = GCN(in_dim = data.x.size(1),hidden_dim = 6,out_dim = dataset.num_classes).to(device)

X = data.x.to(device)
A_norm = A_norm.to(device)
labels = data.y.to(device)
train_mask = data.train_mask.to(device)
test_mask = data.test_mask.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3, weight_decay = 5e-2)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(3000):
  model.train()
  optimizer.zero_grad()

  out = model(A_norm, X)
  loss = loss_fn(out[train_mask], labels[train_mask])

  loss.backward()
  optimizer.step()

  if epoch % 20 == 0:
    pred = out.argmax(dim=1)
    acc = (pred[train_mask] == labels[train_mask]).float().mean()
    print(f"Epoch {epoch} | Loss {loss:.4f} | Train Acc {acc:.3f}")

Epoch 0 | Loss 26.0368 | Train Acc 0.121
Epoch 20 | Loss 21.9397 | Train Acc 0.121
Epoch 40 | Loss 18.6037 | Train Acc 0.129
Epoch 60 | Loss 15.9398 | Train Acc 0.136
Epoch 80 | Loss 13.8123 | Train Acc 0.150
Epoch 100 | Loss 12.0653 | Train Acc 0.164
Epoch 120 | Loss 10.6314 | Train Acc 0.164
Epoch 140 | Loss 9.4499 | Train Acc 0.164
Epoch 160 | Loss 8.4731 | Train Acc 0.171
Epoch 180 | Loss 7.6605 | Train Acc 0.171
Epoch 200 | Loss 6.9681 | Train Acc 0.179
Epoch 220 | Loss 6.3824 | Train Acc 0.193
Epoch 240 | Loss 5.8784 | Train Acc 0.193
Epoch 260 | Loss 5.4322 | Train Acc 0.186
Epoch 280 | Loss 5.0365 | Train Acc 0.186
Epoch 300 | Loss 4.6873 | Train Acc 0.179
Epoch 320 | Loss 4.3778 | Train Acc 0.179
Epoch 340 | Loss 4.1010 | Train Acc 0.179
Epoch 360 | Loss 3.8538 | Train Acc 0.200
Epoch 380 | Loss 3.6324 | Train Acc 0.200
Epoch 400 | Loss 3.4345 | Train Acc 0.214
Epoch 420 | Loss 3.2563 | Train Acc 0.214
Epoch 440 | Loss 3.0967 | Train Acc 0.214
Epoch 460 | Loss 2.9559 | Train A

**Eval loop**

In [34]:
model.eval()
with torch.no_grad():
    out = model(A_norm, X)
    pred = out.argmax(dim=1)
    test_acc = (pred[test_mask] == labels[test_mask]).float().mean()
    print("Test Accuracy:", test_acc.item())


Test Accuracy: 0.7160000205039978


In [ ]:
torch.save(model.state_dict(), "gcn_cora.pth")

In [37]:
model = GCN(in_dim = data.x.size(1),hidden_dim = 6,out_dim = dataset.num_classes).to(device)
model.load_state_dict(torch.load("gcn_cora.pth"))
model.eval()


GCN()